# Desenvolvimento Rápido de Aplicações em Python
### Conversão das Apresentações (2025_05, 2025_06, 2025_07)
**Prof. Raphael M. S. de Jesus**

Este notebook reúne o conteúdo apresentado nas aulas sobre Python, banco de dados SQLite, uso de dataclass, ORM, logger e padrões de projeto.
O material foi reorganizado de forma didática e profissional, com explicações completas e códigos funcionais.

## Conectando ao Banco de Dados SQLite

Para iniciar a interação com um banco de dados SQLite, usamos o módulo `sqlite3`.
A seguir, um exemplo simples de como conectar-se a um banco e fechar a conexão.

In [ ]:
import sqlite3

conexao = sqlite3.connect("banco.sqlite")
print("Conectado com sucesso.")
conexao.close()

## Utilizando `@dataclass` para Modelagem de Dados

O `dataclass` foi introduzido no Python 3.7 e facilita a criação de classes que armazenam dados. Ele automaticamente adiciona métodos especiais como `__init__`, `__repr__` e `__eq__`.

In [ ]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Pessoa:
    cpf: int
    nome: str
    nascimento: date
    oculos: bool

In [ ]:
@dataclass
class Marca:
    id: int
    nome: str
    sigla: str

In [ ]:
@dataclass
class Veiculo:
    placa: str
    cor: str
    proprietario: Pessoa
    marca: Marca

### Criando e exibindo objetos `Pessoa`, `Marca` e `Veiculo`:

In [ ]:
pessoa1 = Pessoa(cpf=12345678900, nome="Raphael", nascimento=date(1984, 7, 26), oculos=True)
marca1 = Marca(id=1, nome="Fiat", sigla="FIA")
veiculo1 = Veiculo(placa="LRW1I27", cor="Cinza", proprietario=pessoa1, marca=marca1)

print(pessoa1)
print(marca1)
print(veiculo1)

## Criando Tabelas SQLite com Segurança

In [ ]:
def criar_tabela_pessoa(conn):
    try:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Pessoa (
                cpf INTEGER PRIMARY KEY,
                nome TEXT NOT NULL,
                nascimento DATE,
                oculos BOOLEAN
            )
        ''')
        conn.commit()
        print("Tabela Pessoa criada com sucesso.")
    except sqlite3.Error as e:
        print(f"Erro ao criar tabela Pessoa: {e}")

## Usando `logger` em vez de `print`

Utilizar `logging` permite registrar erros com mais controle, incluindo data, tipo e mensagem.

In [ ]:
import logging

logging.basicConfig(
    filename="app.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log_info(msg): logging.info(msg)
def log_error(msg): logging.error(msg)

### Inserindo uma Pessoa com logger

In [ ]:
def inserir_pessoa(conn, pessoa: Pessoa):
    try:
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO Pessoa VALUES (?, ?, ?, ?)",
            (pessoa.cpf, pessoa.nome, pessoa.nascimento.isoformat(), pessoa.oculos)
        )
        conn.commit()
        log_info("Pessoa inserida com sucesso.")
    except sqlite3.Error as e:
        log_error(f"Erro ao inserir pessoa: {e}")
        raise

### Atualizando uma Pessoa com logger

In [ ]:
def atualizar_pessoa(conn, pessoa: Pessoa):
    try:
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE Pessoa SET nome=?, nascimento=?, oculos=? WHERE cpf=?
        ''', (pessoa.nome, pessoa.nascimento.isoformat(), pessoa.oculos, pessoa.cpf))
        conn.commit()
        log_info("Pessoa atualizada com sucesso.")
    except sqlite3.Error as e:
        log_error(f"Erro ao atualizar pessoa: {e}")
        raise

### Deletando uma Pessoa

In [ ]:
def apagar_pessoa(conn, cpf: int):
    try:
        cursor = conn.cursor()
        cursor.execute("DELETE FROM Pessoa WHERE cpf=?", (cpf,))
        conn.commit()
        log_info(f"Pessoa com CPF {cpf} deletada com sucesso.")
    except sqlite3.Error as e:
        log_error(f"Erro ao apagar pessoa: {e}")
        raise

## Introdução ao SQLAlchemy (ORM)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///:memory:', echo=True)
Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nome = Column(String)
    idade = Column(Integer)

    def __repr__(self):
        return f"Usuario(id={self.id}, nome={self.nome}, idade={self.idade})"

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Inserindo dados
session.add(Usuario(nome="Raphael", idade=40))
session.commit()

# Consulta
usuarios = session.query(Usuario).all()
for u in usuarios:
    print(u)

## Conclusão

Este material abordou os principais tópicos para o desenvolvimento rápido de aplicações com Python:

- Uso de `dataclass` para modelagem de dados
- Conexão com banco SQLite
- Criação de tabelas e manipulação com segurança
- Uso do `logger` para registrar eventos
- Aplicação de um ORM (SQLAlchemy)

📚 **Próximo passo sugerido**: modularizar o projeto com MVC, dividir código em arquivos como `model.py`, `controller.py`, `view.py` e aplicar os conceitos aprendidos.